In [6]:
import requests
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

API_KEY = input("Por favor, introduzca su API Key aquí (sin comillas): ")

def extraccion_estacion(provincia, ciudad):

    url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones"
    response = requests.get(url, headers={"api_key": API_KEY})

    info = response.json()
    data_url = info["datos"]
    dataset = requests.get(data_url).json()  
    
    df = pd.DataFrame(dataset)
    mask = (df["provincia"] == provincia.upper()) & (df["nombre"] == ciudad.upper())
    df = df[mask]

    return df

extraccion_estacion("BARCELONA", "BARCELONA AEROPUERTO")

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
157,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,08181,020412E


In [7]:
import requests

def extraccion_años(ano_ini, ano_fin, estacion):

    url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/mensualesanuales/datos/anioini/{ano_ini}/aniofin/{ano_fin}/estacion/{estacion}"

    response = requests.get(url, headers={"api_key": API_KEY})

    info = response.json()
    data_url = info["datos"]
    dataset = requests.get(data_url).json()  
 
    df = pd.DataFrame(dataset)

    col_interes = ["indicativo","fecha", "ta_max", "ta_min", "n_llu","w_racha", "hr","p_mes"]
    
    df_clean = df[col_interes].copy()

    df_clean.rename(columns={
        "ta_min": "Temperatura Minima", 
        "ta_max": "Temperatura Maxima",
        "n_llu": "Cantidad dias de lluvia",
        "w_racha": "velocidad del viento",
        "hr": "Humedad Relativa",
        "p_mes": "Precipitaciones mensuales"

    }, inplace=True)

    # convertir fecha
    df_clean["fecha"] = pd.to_datetime(df_clean["fecha"], errors="coerce")
    df_clean = df_clean.sort_values("fecha").reset_index(drop=True)

    # limpiar valores con paréntesis
    def limpiar_parentesis(valor):
        if isinstance(valor, str) and "(" in valor:
            return valor.split("(")[0].strip()
        return valor

    columnas_limpiar = ["Temperatura Maxima", "Temperatura Minima"]
    for col in columnas_limpiar:
        df_clean[col] = df_clean[col].apply(limpiar_parentesis)

   
    def limpiar_w_racha(valor):
        if isinstance(valor, str) and "/" in valor:
            return valor.split("/")[0].strip()
        return valor

    df_clean["velocidad del viento"] = df_clean["velocidad del viento"].apply(limpiar_w_racha)

    # convertir a numérico
    df_clean["Temperatura Maxima"] = pd.to_numeric(df_clean["Temperatura Maxima"], errors="coerce")
    df_clean["Temperatura Minima"] = pd.to_numeric(df_clean["Temperatura Minima"], errors="coerce")
    df_clean["Cantidad dias de lluvia"] = pd.to_numeric(df_clean["Cantidad dias de lluvia"], errors="coerce")
    df_clean["velocidad del viento"] = pd.to_numeric(df_clean["velocidad del viento"], errors="coerce")
    df_clean["Humedad Relativa"] = pd.to_numeric(df_clean["Humedad Relativa"], errors="coerce")
    df_clean["Precipitaciones mensuales"] = pd.to_numeric(df_clean["Precipitaciones mensuales"], errors="coerce")

    return df_clean

años = [(2005,2008),(2009,2011),(2012,2015),(2016,2019),(2020,2023),(2024,2024)]
estacion = "0076"

resultados = [extraccion_años(ini, fin, estacion) for ini, fin in años]



for i, df in enumerate(resultados):
    
    df_clean = pd.concat(resultados,ignore_index=True)
    pd.set_option('display.max_rows', None)    
    pd.set_option('display.max_columns', None)  
    pd.set_option('display.width', None)        
    pd.set_option('display.max_colwidth', None) 
    df_clean

df_clean = df_clean.dropna(subset=["fecha"])
df_clean
df_clean = df_clean.interpolate().round(2)
df_clean

C:\Users\ignac\AppData\Local\Temp\ipykernel_26860\750220835.py:79: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean = df_clean.interpolate().round(2)


,indicativo,fecha,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales
0,0076,2005-01-01,18.70,-1.10,0.00,33.00,61.00,0.00
1,0076,2005-02-01,20.10,0.40,9.00,30.00,61.00,25.20
2,0076,2005-03-01,23.00,0.00,6.00,34.00,70.00,17.20
3,0076,2005-04-01,23.20,7.40,2.00,8.00,65.00,4.20
4,0076,2005-05-01,26.30,11.70,12.00,22.00,63.00,38.20
5,0076,2005-06-01,34.90,15.40,5.00,23.00,64.00,10.00
6,0076,2005-07-01,30.90,18.40,9.00,12.00,63.00,30.90
7,0076,2005-08-01,29.90,14.20,11.00,36.00,61.00,94.10
8,0076,2005-09-01,30.00,12.30,13.00,7.00,65.00,164.90
9,0076,2005-10-01,26.00,11.70,13.00,16.33,73.00,48.30


In [8]:
directory_path = r"..\Desktop\Proyecto AEMET\Data"
os.makedirs(directory_path, exist_ok=True)

df_clean.to_csv(r"..\Desktop\Proyecto AEMET\Data\barcelona.csv", index=False)

In [9]:
 # valores medios
def valores_medios(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.mean(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_medios = valores_medios(df_clean)
df_valores_medios

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,26.0,7.0,14.0,28.0,63.0,76.0,2005-2008
1,27.0,7.0,10.0,31.0,63.0,64.0,2009-2011
2,26.0,8.0,11.0,33.0,68.0,63.0,2012-2015
3,26.0,8.0,12.0,27.0,67.0,84.0,2016-2019
4,26.0,9.0,11.0,27.0,69.0,75.0,2020-2023
5,27.0,9.0,15.0,41.0,69.0,113.0,2024-2024


In [10]:
 # valores maximos
def valores_maximos(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.max(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_maximos = valores_maximos(df_clean)
df_valores_maximos

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,35.0,21.0,107.0,99.0,73.0,581.0,2005-2008
1,37.0,20.0,89.0,99.0,68.0,700.0,2009-2011
2,34.0,21.0,93.0,99.0,80.0,590.0,2012-2015
3,37.0,20.0,110.0,99.0,78.0,753.0,2016-2019
4,35.0,20.0,86.0,99.0,79.0,1012.0,2020-2023
5,34.0,20.0,98.0,99.0,76.0,737.0,2024-2024


In [11]:
 # valores minimos
def valores_minimos(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.min(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_minimos = valores_minimos(df_clean)
df_valores_minimos

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,17.0,-2.0,0.0,2.0,54.0,0.0,2005-2008
1,16.0,-3.0,1.0,2.0,58.0,0.0,2009-2011
2,18.0,-4.0,2.0,1.0,49.0,0.0,2012-2015
3,17.0,-1.0,1.0,1.0,60.0,0.0,2016-2019
4,18.0,-1.0,1.0,7.0,59.0,1.0,2020-2023
5,21.0,2.0,4.0,1.0,65.0,10.0,2024-2024


In [12]:
df_valores_medios.to_csv("../Desktop/Proyecto AEMET/Data/barcelona_medios.csv", index = False)
df_valores_maximos.to_csv("../Desktop/Proyecto AEMET/Data/barcelona_maximos.csv" , index = False)
df_valores_minimos.to_csv("../Desktop/Proyecto AEMET/Data/barcelona_minimos.csv" , index = False)